In [2]:
import optuna
import pandas as pd
from core.backtester import walk_forward
from core.helper import calc_cagr, calc_sharpe, calc_volatility, calc_sortino


def objective(trial, tickers, start_invest , end_invest, start_capital):

  lookback_months = trial.suggest_int("lookback_months", 1, 24)
  skip_months = trial.suggest_int("skip_months", 0, 6)
  top_k = trial.suggest_int("top_k", 1, 10)
  stop_loss = trial.suggest_float("stop_loss", 0,  0.5)


  try: 
    trade_records, capital, ben_capital, str_mon_returns, ben_mon_returns, \
        portfolio_value, benchmark_value, max_drawdown, max_drawdown_ben, \
        total_trades, wins = walk_forward(
            tickers, start_invest, end_invest, lookback_months, 
            skip_months, start_capital, stop_loss
        )
      
    sharpe = calc_sharpe(str_mon_returns, 0.04)
    cagr = calc_cagr(start_capital , capital, str_mon_returns, start_invest, end_invest)
    volatility = calc_volatility(str_mon_returns)
    sortino = calc_sortino(str_mon_returns, 0.04)

    win_rate = (wins / total_trades) if total_trades > 0 else 0

    score = sharpe*(1-max_drawdown) + (1+cagr/10)*(1+win_rate/2)

    if np.isnan(score or np.isinf(score)):
      return -1000
    
    return score

  except Exception as e:
    print(f"Error in trial:{e}")
    return -1000



In [ ]:
def optimize_strategy(tickers, start_invest, end_invest, start_capital, n_trials=100, direction="maximize", metric="composite"):

  study = optuna.create_study(direction=direction)

  if metric == "sharpe":
    def objective_sharpe(trial):
      return objective(trial, tickers, start_invest, end_invest, start_capital)
    study.optimize(objective_sharpe, n_trials=n_trials)
  
  elif metric == "cagr":
    def objective_cagr(trial):
      return objective(trial, tickers, start_invest, end_invest, start_capital, metric="cagr")
    study.optimize(objective_cagr, n_trials=n_trials)
  
  else:
    study.optimize(lambda trial: objective(trial, tickers, start_invest, end_invest, start_capital), n_trials=n_trials
    )
  return study


In [6]:
print("===Basic Optuna Example===")

def simple_objective(trial):
  x = trial.suggest_float("x", -10, 10)
  y= trial.suggest_float("y", -10, 10)
  return (x-2)**2 + (y-3)**2

study = optuna.create_study(direction="minimize")
study.optimize(simple_objective , n_trials=200)

print(f"Best value: {study.best_value}")
print(f"Best params: {study.best_params}")
print(f"Number of trials: {len(study.trials)}")


[I 2025-10-08 21:24:13,778] A new study created in memory with name: no-name-763cf2e3-f8cb-4d7e-84da-f8d62fb7d7d9
[I 2025-10-08 21:24:13,784] Trial 0 finished with value: 135.0272461436398 and parameters: {'x': -9.343511803767292, 'y': 0.47968531698284167}. Best is trial 0 with value: 135.0272461436398.
[I 2025-10-08 21:24:13,785] Trial 1 finished with value: 39.6209194236732 and parameters: {'x': 8.110454270684617, 'y': 1.4889513477239298}. Best is trial 1 with value: 39.6209194236732.
[I 2025-10-08 21:24:13,785] Trial 2 finished with value: 157.44327976815595 and parameters: {'x': 8.133567712117756, 'y': -7.946352218388668}. Best is trial 1 with value: 39.6209194236732.
[I 2025-10-08 21:24:13,785] Trial 3 finished with value: 20.3177959016844 and parameters: {'x': 5.704899305803048, 'y': 5.567394990168925}. Best is trial 3 with value: 20.3177959016844.
[I 2025-10-08 21:24:13,785] Trial 4 finished with value: 46.96063922039718 and parameters: {'x': 0.63728601838217, 'y': -3.7159250907

===Basic Optuna Example===


[I 2025-10-08 21:24:13,980] Trial 59 finished with value: 34.411731764369534 and parameters: {'x': 5.970186863831319, 'y': 7.318489091178854}. Best is trial 53 with value: 0.817937920505935.
[I 2025-10-08 21:24:13,985] Trial 60 finished with value: 11.10376026559997 and parameters: {'x': 4.2897475644876435, 'y': 5.42091229839556}. Best is trial 53 with value: 0.817937920505935.
[I 2025-10-08 21:24:13,989] Trial 61 finished with value: 2.101783428302931 and parameters: {'x': 2.9337888294441123, 'y': 4.108973331648839}. Best is trial 53 with value: 0.817937920505935.
[I 2025-10-08 21:24:14,006] Trial 62 finished with value: 1.321655458635675 and parameters: {'x': 3.1467141452627514, 'y': 3.081866523622251}. Best is trial 53 with value: 0.817937920505935.
[I 2025-10-08 21:24:14,029] Trial 63 finished with value: 9.95282520150124 and parameters: {'x': 5.132771954159238, 'y': 3.372243313888308}. Best is trial 53 with value: 0.817937920505935.
[I 2025-10-08 21:24:14,054] Trial 64 finished wi

Best value: 0.03051037097534606
Best params: {'x': 1.831522818175123, 'y': 3.046106509081619}
Number of trials: 200


In [7]:
# Test 2: Test Your Objective Function with Fixed Parameters
print("=== Test 2: Test Your Objective Function ===")

# Create a mock trial object for testing
class MockTrial:
    def __init__(self, params):
        self.params = params
    
    def suggest_int(self, name, low, high):
        return self.params.get(name, (low + high) // 2)
    
    def suggest_float(self, name, low, high):
        return self.params.get(name, (low + high) / 2)

# Test with some sample parameters
test_params = {
    "lookback_months": 6,
    "skip_months": 1,
    "top_k": 2,
    "stop_loss": 0.1
}

mock_trial = MockTrial(test_params)

# Test your objective function
tickers = "AAPL,MSFT,AMZN"
start_invest = "2022-01-01"
end_invest = "2023-01-01"
start_capital = 10000

print("Testing objective function...")
try:
    score = objective(mock_trial, tickers, start_invest, end_invest, start_capital)
    print(f"Test score: {score}")
    print("✅ Objective function works!")
except Exception as e:
    print(f"❌ Error in objective function: {e}")

=== Test 2: Test Your Objective Function ===
Testing objective function...


[*********************100%***********************]  3 of 3 completed

Ticker            AMZN                                                  \
Price             Open        High         Low       Close   Adj Close   
Date                                                                     
2021-04-01  155.897003  158.121994  155.777496  158.050003  158.050003   
2021-04-05  158.649994  161.798004  158.061996  161.336502  161.336502   
2021-04-06  161.187500  162.365494  160.852005  161.190994  161.190994   
2021-04-07  161.690002  165.180496  161.182495  163.969498  163.969498   
2021-04-08  165.544998  166.225006  164.600006  164.964996  164.964996   
...                ...         ...         ...         ...         ...   
2023-02-22   95.099998   97.010002   94.800003   95.790001   95.790001   
2023-02-23   96.120003   96.430000   93.669998   95.820000   95.820000   
2023-02-24   93.529999   94.139999   92.320000   93.500000   93.500000   
2023-02-27   94.279999   94.779999   93.139999   93.760002   93.760002   
2023-02-28   93.139999   94.690002   9


/Users/jaykalaria/portfolio/momentum-market/core/helper.py:65: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data= raw_data.fillna(method="ffill").fillna(method="bfill")
